In [1]:
import json
import pandas as pd
import numpy as np
import random
import pickle
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['>','!,','.',',']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\elirj\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


[(['hello'], 'greetings'), (['hi'], 'greetings'), (['hey'], 'greetings'), (['hola'], 'greetings'), (['good', 'day'], 'greetings'), (['salutations'], 'greetings'), (['Yoo'], 'greetings'), (['cya'], 'goodbye'), (['See', 'you', 'Later'], 'goodbye'), (['take', 'care'], 'goodbye'), (['hola'], 'goodbye'), (['good', 'day'], 'goodbye'), (['How', 'old', 'are', 'you', '?'], 'age'), (['What', 'is', 'your', 'age', '?'], 'age'), (['What', 'year', 'were', 'you', 'born', '?'], 'age'), (['May', 'I', 'ask', 'who', 'this', 'is', '?'], 'name'), (['Who', 'are', 'you', '?'], 'name'), (['Can', 'you', 'please', 'tell', 'me', 'your', 'name', '?'], 'name'), (['Who', 'are', 'you', '?'], 'name'), (['good', 'day'], 'name'), (['salutations'], 'name'), ([], 'name'), (['headache'], 'symptoms'), (['tiredness'], 'symptoms'), (['flu'], 'symptoms'), (['vomit'], 'symptoms'), (['fever'], 'symptoms'), (['diarrhea'], 'symptoms'), (['How', 'are', 'you', 'feeling', 'today', '?'], 'diagnosis'), (['What', 'seems', 'to', 'be', '

In [2]:

# import json
# intents = json.loads(open('intents.json', 'r+').read())
# print(json.dumps(intents, indent=4))

In [3]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
    
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print(words)

['?', 'Am', 'And', 'Can', 'How', 'I', 'Later', 'May', 'See', 'What', 'Who', 'Yoo', 'about', 'age', 'are', 'ask', 'be', 'bleeding', 'bone', 'born', 'broken', 'care', 'come', 'concussion', 'contagious', 'covid', 'cya', 'day', 'diarrhea', 'feeling', 'fever', 'flu', 'good', 'have', 'headache', 'hello', 'here', 'hey', 'hi', 'hola', 'impact', 'is', 'me', 'name', 'old', 'please', 'problem', 'rash', 'salutation', 'seems', 'skin', 'sore', 'swelling', 'take', 'tell', 'the', 'this', 'tiredness', 'to', 'today', 'virus', 'vomit', 'were', 'who', 'why', 'will', 'wound', 'year', 'you', 'your']


C:\Users\elirj\anaconda3\lib\site-packages\ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [4]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))


In [5]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(np.array(train_x), np.array(train_y), epochs=10, batch_size=5,)
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("model.h5", hist)
print("Done")

Epoch 1/10
10/10 [==============================] - 0s 500us/step - loss: 2.2397 - accuracy: 0.1042
Epoch 2/10
10/10 [==============================] - 0s 500us/step - loss: 2.1634 - accuracy: 0.1667
Epoch 3/10
10/10 [==============================] - 0s 500us/step - loss: 2.1310 - accuracy: 0.1458
Epoch 4/10
10/10 [==============================] - 0s 500us/step - loss: 2.0519 - accuracy: 0.2917
Epoch 5/10
10/10 [==============================] - 0s 400us/step - loss: 2.0203 - accuracy: 0.2708
Epoch 6/10
10/10 [==============================] - 0s 500us/step - loss: 2.0382 - accuracy: 0.1458
Epoch 7/10
10/10 [==============================] - 0s 400us/step - loss: 2.0076 - accuracy: 0.2083
Epoch 8/10
10/10 [==============================] - 0s 500us/step - loss: 1.9611 - accuracy: 0.1875
Epoch 9/10
10/10 [==============================] - 0s 400us/step - loss: 1.9020 - accuracy: 0.3542
Epoch 10/10
10/10 [==============================] - 0s 400us/step - loss: 1.7684 - accuracy: 0.2917

10/10 [==============================] - 0s 500us/step - loss: 0.5373 - accuracy: 0.7917
Epoch 72/200
10/10 [==============================] - 0s 500us/step - loss: 0.6641 - accuracy: 0.6250
Epoch 73/200
10/10 [==============================] - 0s 500us/step - loss: 0.6577 - accuracy: 0.6875
Epoch 74/200
10/10 [==============================] - 0s 500us/step - loss: 0.5321 - accuracy: 0.6667
Epoch 75/200
10/10 [==============================] - 0s 500us/step - loss: 0.5539 - accuracy: 0.7500
Epoch 76/200
10/10 [==============================] - 0s 500us/step - loss: 0.5873 - accuracy: 0.7500
Epoch 77/200
10/10 [==============================] - 0s 500us/step - loss: 0.5409 - accuracy: 0.7083
Epoch 78/200
10/10 [==============================] - 0s 500us/step - loss: 0.5828 - accuracy: 0.7500
Epoch 79/200
10/10 [==============================] - 0s 500us/step - loss: 0.6336 - accuracy: 0.6667
Epoch 80/200
10/10 [==============================] - 0s 500us/step - loss: 0.5424 - accuracy: 

Epoch 151/200
10/10 [==============================] - 0s 500us/step - loss: 0.4671 - accuracy: 0.6875
Epoch 152/200
10/10 [==============================] - 0s 500us/step - loss: 0.4404 - accuracy: 0.7500
Epoch 153/200
10/10 [==============================] - 0s 500us/step - loss: 0.4882 - accuracy: 0.7292
Epoch 154/200
10/10 [==============================] - 0s 500us/step - loss: 0.5281 - accuracy: 0.7292
Epoch 155/200
10/10 [==============================] - 0s 500us/step - loss: 0.4826 - accuracy: 0.6875
Epoch 156/200
10/10 [==============================] - 0s 500us/step - loss: 0.4703 - accuracy: 0.6875
Epoch 157/200
10/10 [==============================] - 0s 500us/step - loss: 0.4506 - accuracy: 0.7083
Epoch 158/200
10/10 [==============================] - 0s 400us/step - loss: 0.4341 - accuracy: 0.7917
Epoch 159/200
10/10 [==============================] - 0s 500us/step - loss: 0.5036 - accuracy: 0.6875
Epoch 160/200
10/10 [==============================] - 0s 400us/step - lo